In [3]:
!ls

nba_team_stats_17-18.csv
nba_team_stats_17-18.xlsx
project.ipynb


In [1]:
import pandas as p

trainMatchData = p.read_csv('../matchups/matchups16-17.csv')
trainMatchData.head()

,Visitor,PTS,Home,PTS.1,visitorWin,homeWin
0,Boston Celtics,99,Cleveland Cavaliers,102,NaN,NaN
1,Houston Rockets,122,Golden State Warriors,121,NaN,NaN
2,Milwaukee Bucks,108,Boston Celtics,100,NaN,NaN
3,Atlanta Hawks,117,Dallas Mavericks,111,NaN,NaN
4,Charlotte Hornets,90,Detroit Pistons,102,NaN,NaN


In [36]:
for i,row in trainMatchData.iterrows():
    vpts = row['PTS']
    hpts = row['PTS.1']
    vwin = 0
    hwin = 0
    print(row['PTS'])
    print(row['PTS.1'])
    if (vpts > hpts):
        vwin = 1
    else:
        hwin = 1
    trainMatchData.at[i,'visitorWin'] = vwin
    trainMatchData.at[i,'homeWin'] = hwin
# end for

99
102

122
121

108
100

117
111

90
102

131
140

91
103

109
116

124
76

105
100



In [55]:
import pandas as p
import vars

TrainMatchFile = '../matchups/matchups16-17.csv'
TestMatchFile = '../matchups/matchups17-18.csv'
def labelWinners(filename):
    MatchData = p.read_csv(filename)

    for i,row in MatchData.iterrows():
        vpts = row['PTS']
        hpts = row['PTS.1']
        vwin = 0
        hwin = 0
        if (vpts > hpts):
            vwin = 1
        else:
            hwin = 1
        MatchData.at[i,'visitorWin'] = vwin
        MatchData.at[i,'homeWin'] = hwin
    filename = filename[:-4] + '-labeled.csv'
    MatchData.to_csv(filename)
    return

def build_against_dict(filenameList,year):
    data = dict()
    for fname in filenameList:
        key = fname[5:-4]
        fname = '../data' + year + '/' + fname
        data[key] = p.read_csv(fname)
    return data

def build_year_long_dict(filenameList):
    data = dict()
    data['16-17'] = p.read_csv(filenameList[0])
    data['17-18'] = p.read_csv(filenameList[1])
    return data

def build_match_data_dict():
    data = dict()
    data['train'] = p.read_csv('../matchups/matchups16-17-labeled.csv')
    data['test'] = p.read_csv('../matchups/matchups17-18-labeled.csv')
    return data

def add_classifiers(raw_match_data,year_long_data,teams_against_data):
    final_data = raw_match_data
    c = 0
    for i,row in raw_match_data.iterrows():
        vteam = row['Visitor']
        hteam = row['Home']
        vTotWL = get_stat(year_long_data,'TEAM', vteam, 'WIN%')
        hTotWL = get_stat(year_long_data,'TEAM', hteam, 'WIN%')
        visvisWL = calcVisitorWL(vteam)
        homehomeWL = calcHomeWL(hteam)
        vMatchupWL = get_stat(teams_against_data[vars.nameMapping[hteam]], 'TEAM', vteam, 'WIN%')
        hMatchupWL = get_stat(teams_against_data[vars.nameMapping[vteam]], 'TEAM', hteam, 'WIN%')
        # print("visitors: " + vteam)
        # print(vMatchupWL)
        # print("home: " + hteam)
        # print(hMatchupWL)
        final_data.at[i,'visTotWL'] = vTotWL
        final_data.at[i,'homeTotWL'] = hTotWL
        final_data.at[i,'visvisWL'] = visvisWL
        final_data.at[i,'homehomeWL'] = homehomeWL
        final_data.at[i,'visMatchupWL'] = vMatchupWL
        final_data.at[i,'homeMatchupWL'] = hMatchupWL
        final_data['winner'] = ''
        if (row['visitorWin']):
            final_data.at[i, 'winner'] = vteam
        else:
            final_data.at[i, 'winner'] = hteam
        c += 1
        if (c == 2):
            return final_data
    return final_data

def get_stat(frame, compCol, compVal, tarVal):
    return frame.loc[frame[compCol] == compVal][tarVal]

def calcVisitorWL(visitingTeam):
    visitorOutcomes = get_stat(matchup_data['train'], 'Visitor', visitingTeam, 'visitorWin')
    visitorWinsCount = visitorOutcomes.sum()
    #print("visiting wins: " + str(visitorWinsCount))
    #losses = visitorOutcomes.size - visitorWinsCount
    return (visitorWinsCount / visitorOutcomes.size)

def calcHomeWL(homeTeam):
    homeOutcomes = get_stat(matchup_data['train'], 'Home', homeTeam, 'homeWin')
    homeWinsCount = homeOutcomes.sum()
    #print('home wins: ' + str(homeWinsCount))
    return (homeWinsCount / homeOutcomes.size)

def main():
    teams_against_data = dict()
    teams_against_data['16-17'] = build_against_dict(vars.teamNames,'16-17')
    teams_against_data['17-18'] = build_against_dict(vars.teamNames,'17-18')
    year_long_data = build_year_long_dict(vars.yearFiles)
    final_data = add_classifiers(matchup_data['train'], year_long_data['16-17'], teams_against_data['16-17'])
    print(final_data.head())
    # How to get Pistons year long win%
    #print(get_stat(year_long_data['16-17'],'TEAM', 'Detroit Pistons', 'FG%'))
    # how to get celtics w/l% against cavs
    #print(get_stat(teams_against_data['16-17']['Celtics'], 'TEAM', 'Cleveland Cavaliers', 'WIN%' ))
    # this is total win% ... use this or just grab the value!
    #print((calcVisitorWL('Detroit Pistons') + calcHomeWL('Detroit Pistons'))/2)

#labelWinners(TrainMatchFile)
#labelWinners(TestMatchFile)
matchup_data = build_match_data_dict()
main()


AttributeError: module 'vars' has no attribute 'nameMapping'

In [66]:
fdtest = p.read_csv('finaldata-test.csv')
fdtest.head()

,Unnamed: 0,Unnamed: 0.1,Visitor,Home,visTotWL,homeTotWL,visvisWL,homehomeWL,visMatchupWL,homeMatchupWL,winner
0,0,0,New York Knicks,Cleveland Cavaliers,0.378,0.622,0.292683,0.756098,0.000,1.000,1
1,1,1,San Antonio Spurs,Golden State Warriors,0.744,0.817,0.731707,0.878049,0.667,0.333,0
2,2,2,Utah Jazz,Portland Trail Blazers,0.622,0.500,0.536585,0.609756,0.500,0.500,1
3,3,3,Brooklyn Nets,Boston Celtics,0.244,0.646,0.170732,0.731707,0.000,1.000,1
4,4,4,Dallas Mavericks,Indiana Pacers,0.402,0.512,0.292683,0.707317,0.500,0.500,1


In [64]:
year_long_data = build_year_long_dict(vars.yearFiles)
get_stat(year_long_data['16-17'], 'TEAM', 'Los Angeles Clippers', 'WIN%')

5    0.622
Name: WIN%, dtype: float64